In [2]:
!which python

/Users/hosjiu/workspace/coding/rag/AutumnRAG/.venv/bin/python


In [3]:
### Indexing
# 1. Load Documents
from langchain_community.document_loaders import Docx2txtLoader

data_path = "../data/kht_contract.docx"
word_loader = Docx2txtLoader(data_path)
documents = word_loader.load()

print(documents)

[Document(metadata={'source': '../data/kht_contract.docx'}, page_content='GARAGE OTO HUY HUNG \n\nCÔNG TY TNHH MTV KIM HUY THÀNH\n\nTổ 58 Thanh Vinh 1, Hòa Khánh Bắc, LiênChiểu, Đà Nẵng\n\n\n\nCỘNG HÒA XÃ HÔI CHỦ NGHĨA VIỆT NAM\n\nĐộc lập – Tự do – Hạnh phúc\n\n______o0o______\n\nHỢP ĐỒNG SỬA CHỮA XE\n\nCăn cứ Bộ luật thương mại nước CHXHCN Việt Nam số 33/2005/QH11 được Quốc hội thông qua ngày 14/6/2005\n\nCăn cứ vào nhu cầu và năng lực của hai bên.\n\nHôm nay, ngày ..... tháng ..... năm 2025 tại văn phòng công ty TNHH MTV KIM HUY THÀNH chúng tôi gồm có:\n\n\tĐẠI DIỆN BÊN A\t: CÔNG TY TNHH THƯƠNG MẠI VÀ DỊCH VỤ HIỀN HUY\n\n\t\tĐại diện\t: Huỳnh Văn Chỉnh\tChức vụ: Giám đốc\n\n\t\tĐịa chỉ\t: Thôn Phước Hưng, Xã Hoà Nhơn, H. Hoà Vang, Tp. Đà Nẵng, Việt Nam\n\n\t\tMST\t: 0400979608\n\n\t\tTài khoản\t: 2008201002129\n\n\t\tTại\t: Ngân hàng NN&PTNT Huyện Hoà Vang, thành phố Đà Nẵng.\n\n\t\tĐẠI DIỆN BÊN B\t: CÔNG TY TNHH MTV KIM HUY THÀNH\n\n\t\tĐại diện\t: Dương Thị Kiều Linh\tChức vụ: 

In [4]:
# 2. Text Splitting
# Should split text before feeding it into Text Embedder model to get embedding
# One of advantages of this strategy is to make it work with small context window Language Models
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
splits = text_splitter.split_documents(documents)
print(splits)

[Document(metadata={'source': '../data/kht_contract.docx'}, page_content='GARAGE OTO HUY HUNG \n\nCÔNG TY TNHH MTV KIM HUY THÀNH'), Document(metadata={'source': '../data/kht_contract.docx'}, page_content='Tổ 58 Thanh Vinh 1, Hòa Khánh Bắc, LiênChiểu, Đà Nẵng\n\n\n\nCỘNG HÒA XÃ HÔI CHỦ NGHĨA VIỆT NAM'), Document(metadata={'source': '../data/kht_contract.docx'}, page_content='Độc lập – Tự do – Hạnh phúc\n\n______o0o______\n\nHỢP ĐỒNG SỬA CHỮA XE'), Document(metadata={'source': '../data/kht_contract.docx'}, page_content='Căn cứ Bộ luật thương mại nước CHXHCN Việt Nam số 33/2005/QH11 được Quốc hội thông qua ngày'), Document(metadata={'source': '../data/kht_contract.docx'}, page_content='qua ngày 14/6/2005'), Document(metadata={'source': '../data/kht_contract.docx'}, page_content='Căn cứ vào nhu cầu và năng lực của hai bên.'), Document(metadata={'source': '../data/kht_contract.docx'}, page_content='Hôm nay, ngày ..... tháng ..... năm 2025 tại văn phòng công ty TNHH MTV KIM HUY THÀNH chú

In [5]:
# 3. Embedding Text + Create Vector Store
# Embedding texts and build a vector store from newly generated embedddings for later use
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


text_embedder = HuggingFaceEmbeddings(
    model_name="google/embeddinggemma-300m",
    query_encode_kwargs={"prompt_name": "query"},
    encode_kwargs={"prompt_name": "document"}
)
vector_store = FAISS.from_documents(
    documents=splits,
    embedding=text_embedder,
    distance_strategy="MAX_INNER_PRODUCT" # Must use if our text embedding model is Google's EmbeddingGemma
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
# 4. Information Retrieval
test_query = "Giám đốc của công ty Kim Huy Thành là ai"
top_k = 3
# retrieval_results = vector_store.similarity_search(test_query, k=top_k)
retriever = vector_store.as_retriever(search_kwargs={'k': top_k})
retrieval_results = retriever.invoke(test_query)

for i in range(top_k):
    print(retrieval_results[i])

page_content='Đại diện	: Huỳnh Văn Chỉnh	Chức vụ: Giám đốc' metadata={'source': '../data/kht_contract.docx'}
page_content='Đại diện	: Dương Thị Kiều Linh	Chức vụ: Giám đốc' metadata={'source': '../data/kht_contract.docx'}
page_content='ĐẠI DIỆN BÊN B	: CÔNG TY TNHH MTV KIM HUY THÀNH' metadata={'source': '../data/kht_contract.docx'}


In [18]:
# 5. Generation
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

MODEL_SERVER_URL = "https://92d2avvnzvqndz-8000.proxy.runpod.net" # host LLM, for example, Runpod.

model = ChatOpenAI(
    model="Qwen/Qwen3-4B-Instruct-2507",
    openai_api_base=MODEL_SERVER_URL,
    openai_api_key="EMPTY"
)

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x149a435d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x149b58550>, root_client=<openai.OpenAI object at 0x149a69a50>, root_async_client=<openai.AsyncOpenAI object at 0x149a43810>, model_name='Qwen/Qwen3-4B-Instruct-2507', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://92d2avvnzvqndz-8000.proxy.runpod.net')

In [24]:
TEMPLATE = """Answer the question based on only on the following context
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(TEMPLATE)
chain = prompt | model

chain.invoke({
    'context': retrieval_results,
    'question': 'ai là giám đốc công ty kim huy thành?'
})

NotFoundError: Error code: 404 - {'detail': 'Not Found'}